# Sustainable Farming Chatbot

A sustainable agriculture software project was developed that works with a chatbot. It was developed in Spanish using the SpaCy library to use a lemmatizer for sentence tokens. Based on this, the model can classify possible responses to the user in an intents.JSON file. The project's idea is to create server-side functionality with FLASK, in which an API will be sent to the domain that will host the chatbot interface to resolve pest-related issues. There is an issue with the tokenizer in the server-side chatbot where it only works if the sentence is filled in with the same character format as in the JSON file. Through the Jupyter file, a local interface is available in which the chatbot has corrected this issue, and the tokenizer works without problems.

The chatbot works by querying when the plant has one of the following pests: Bacterial spot, Black measles, Black rot, Early blight, Late blight, Leaf scorch, Rust, Scab, Spot, there is also an option for unknown and no pest.

In [ ]:
import tkinter as tk
from tkinter import *
import nltk
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import tflearn
import numpy as np
import random
import pickle
import json
import os
import spacy
import re

# Cargar modelo de spaCy en español
nlp = spacy.load('es_core_news_sm')

with open('intents.json', encoding='utf-8') as file:
    data = json.load(file)

def lemmatize_sentence(sentence):
    sentence = sentence.lower()
    sentence = re.sub('[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ]', ' ', sentence)
    doc = nlp(sentence)
    lemmas = [token.lemma_ for token in doc]
    print(f"Lemmatized '{sentence}' to {lemmas}")
    return lemmas

if os.path.exists("data.pickle"):
    os.remove("data.pickle")
if os.path.exists("model.tflearn.index"):
    os.remove("model.tflearn.index")
if os.path.exists("model.tflearn.meta"):
    os.remove("model.tflearn.meta")
if os.path.exists("model.tflearn.data-00000-of-00001"):
    os.remove("model.tflearn.data-00000-of-00001")

print("Old files deleted, data processing and training the model from scratch")

words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])
        
        words.extend(lemmatize_sentence(pattern))

    if intent['tag'] not in labels:
        labels.append(intent['tag'])

words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    wrds = lemmatize_sentence(' '.join(doc))

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

training = np.array(training)
output = np.array(output)

with open("data.pickle", "wb") as f:
    pickle.dump((words, labels, training, output), f)

tf.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

def bag_of_words(s, words):
    s = s.lower()
    s = re.sub('[^a-zA-ZáéíóúÁÉÍÓÚñÑüÜ]', ' ', s)
    bag = [0 for _ in range(len(words))]
    s_words = lemmatize_sentence(s)

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1

    print(f"Bag of words for '{s}': {bag}")
    return np.array(bag)

def chatbot_response(msg):
    bow = bag_of_words(msg, words)
    results = model.predict([bow])
    print(f"Model prediction for '{msg}': {results}")
    results_index = np.argmax(results)
    tag = labels[results_index]

    for tg in data['intents']:
        if tg['tag'] == tag:
            responses = tg['responses']
            return random.choice(responses)

    return "I'm sorry, I don't understand what you mean."

base = Tk()  
base.title("Chatbot")  
base.geometry("400x500")  

ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font=("Arial", 12), wrap=WORD)
ChatLog.config(foreground="black")
ChatLog.insert(END, "SALUDOS BIENVENIDO\n\n") 
ChatLog.place(x=6, y=6, height=386, width=370) 

scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
scrollbar.place(x=376, y=6, height=386)

ChatLog.config(state=DISABLED)

EntryBox = Text(base, bd=0, bg="white", width="29", height="5", font=("Arial", 12), wrap=WORD)
EntryBox.place(x=6, y=401, height=90, width=265)

def send(event=None):
    msg = EntryBox.get("1.0", 'end-1c').strip()
    EntryBox.delete("0.0", END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="black")

        res = chatbot_response(msg)
        ChatLog.insert(END, "ChatBOT: " + res + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)

SendButton = Button(base, font=("Verdana", 12, 'bold'), text="Send", width="9",
                   height=5, bd=0, bg="blue", activebackground="gold",
                   fg='#ffffff', command=send)
SendButton.place(x=282, y=401, height=90)

base.bind('<Return>', send)

base.mainloop()


Training Step: 8999  | total loss: 0.00053 | time: 0.027s
| Adam | epoch: 1000 | loss: 0.00053 - acc: 1.0000 -- iter: 64/65
Training Step: 9000  | total loss: 0.00050 | time: 0.029s
| Adam | epoch: 1000 | loss: 0.00050 - acc: 1.0000 -- iter: 65/65
--
INFO:tensorflow:C:\Users\cardo\Documents\Samsung\Chatbot prototipo\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
Lemmatized 'sarampion' to ['sarampion']
Bag of words for 'sarampion': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Model prediction for 'sarampion': [[2.95645342e-10 0.00000000e+00 0.00000000e+00 1.13007505e-29
  0.00000000e+00 8.88616592e-03 1.12237138e-15 0.00000000e+00
  1.32241957e-02 9.77889597e-01 6.65444873e-08 6.87948602e-20]]
Lemmatized 'manchas' to ['mancha']
Bag of words for 'manchas': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0